# Doppler Shift

We take $v_{out}$ with $v_{rot_0} = 0$ km/s. Then we Doppler shift it to reproduce $v_{out}$ with $v_{rot}$. The combinations for each of the 3 taus are:

* $v_{rot} = 50$ km/s and $v_{out} = 25$ km/s
* $v_{rot} = 50$ km/s and $v_{out} = 50$ km/s
* $v_{rot} = 50$ km/s and $v_{out} = 75$ km/s -> data vout=75?
* $v_{rot} = 100$ km/s and $v_{out} = 25$ km/s
* $v_{rot} = 100$ km/s and $v_{out} = 50$ km/s
* $v_{rot} = 100$ km/s and $v_{out} = 75$ km/s -> data vout=75?

This is an example:

![example](../../paper/figures/rotation_doppler_outflow.png)

### Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from mpl_toolkits.mplot3d import Axes3D
import os
plt.rc('text', usetex=True)
plt.rc('font', size=20)

In [33]:
def read_data(vrot=0, vout=5, logtau=5, input_dir="../../data/"):
    tau_name = 'tau10E' + str(logtau)
    vrot_name = 'vrot' + str(vrot)
    vout_name = 'vout' + str(vout)
    filename = tau_name + '_' + vrot_name + '_' + vout_name + '_out.ascii'
    fname =  os.path.join(input_dir, tau_name, vrot_name, vout_name, filename)

    dtype=[('x', 'f8'),('y', 'f8'), ('z', 'f8'),
           ('x_u', 'f8'),('y_u', 'f8'),('z_u', 'f8'),
           ('x_frec', 'f8'), ('escaped', 'i8'), ('n_scattering', 'i8')]

    df = np.loadtxt(fname, skiprows=1, dtype=dtype)
    df = df[df['escaped']==0]
    return df

In [35]:
# We are going to transform data_A to rewrite data_C and compare it against data_B
data_A = read_data(vout=50, vrot=0, logtau=6)
data_B = read_data(vout=50, vrot=100, logtau=6)
data_C = read_data(vout=50, vrot=0, logtau=6)

#data_C = read_data(vout=0, vrot=100, logtau=6)
#../../data/tau10E5/vrot0/vout5/tau10E5_vrot0_vout5_out.ascii

### Functions

In [2]:

# Reading data

def read_data(vrot0, vout, logtau):
    tau_dir = 'tau10E'+str(logtau)
    vrot0_dir = 'vrot'+str(vrot0)
    vout_dir = 'vout'+str(vout)
    
    fname = '../../data/'+tau_dir+'/'+vrot0_dir+'/'+vout_dir+'/'+tau_dir+'_'+vrot0_dir+'_'+vout_dir+'_out.ascii'
    df = pd.read_csv(fname, delimiter=' ', dtype='float64')
        
    escaped = df['escaped']
    x = df['x']
    y = df['y']
    z = df['z']
    x_u = df['x_u']
    y_u = df['y_u']
    z_u = df['z_u']
    x_frec = df['x_frec']

    inds = np.where(escaped == 0)[0]

    x_escaped = np.array(x)[inds]
    y_escaped = np.array(y)[inds]
    z_escaped = np.array(z)[inds]
    x_u_escaped = np.array(x_u)[inds]
    y_u_escaped = np.array(y_u)[inds]
    z_u_escaped = np.array(z_u)[inds]
    x_frec_escaped = np.array(x_frec)[inds]

    return x_escaped, y_escaped, z_escaped, x_u_escaped, y_u_escaped, z_u_escaped, x_frec_escaped

In [3]:
# Auxiliar functions

def get_angles(num_thetas, num_phis):
    '''Get the angles'''
    cos_thetas = np.linspace(0,1,num_thetas+1) 
    cos_thetas_plus = np.linspace(-1,0,num_thetas+1) 

    thetas = np.sort(np.array([np.arccos(ct) for ct in cos_thetas]))
    thetas_plus = np.array([np.arccos(ctp) for ctp in cos_thetas_plus])

    phis = np.linspace(0,2*np.pi,num_phis+1) 
    
    return thetas, thetas_plus, phis

def get_radius(x_escaped, y_escaped, z_escaped):
    '''Get the real positions in the sphere at which the photons escaped.
       TODO calculate real R'''
    distance = np.sqrt(x_escaped**2 + y_escaped**2 + z_escaped**2)
    R = np.min(distance)
    
    return R

def get_final_positions_sphere(x_escaped, y_escaped, z_escaped, x_u_escaped, y_u_escaped, z_u_escaped, R):
    '''Get the real positions in the sphere at which the photons escaped'''
    r_escaped_squared = x_escaped**2 + y_escaped**2 + z_escaped**2
    k_u_escaped_squared = x_u_escaped**2 + y_u_escaped**2 + z_u_escaped**2
    r_escaped_dot_k_u_escaped = x_escaped*x_u_escaped + y_escaped*y_u_escaped + z_escaped*z_u_escaped

    t_a = k_u_escaped_squared
    t_b = 2.0*r_escaped_dot_k_u_escaped
    t_c = r_escaped_squared - R**2

    t_plus = (-t_b + np.sqrt(t_b**2 - 4*t_a*t_c))/(2*t_a)
    t_minus = (-t_b - np.sqrt(t_b**2 - 4*t_a*t_c))/(2*t_a)
    t = []

    for i in range(num_photons_escaped):
        t_p = t_plus[i]
        t_m = t_minus[i]
        t.append(np.amax([t_p, t_m]))

    t = np.array(t)

    x_sphere = x_escaped + x_u_escaped*t
    y_sphere = y_escaped + y_u_escaped*t
    z_sphere = z_escaped + z_u_escaped*t

    return x_sphere, y_sphere, z_sphere

def filter_by_theta(z_u_escaped, x_frec_escaped, theta_lower, theta_upper, theta_plus_lower, theta_plus_upper):
    '''Return only the x_frec_escaped between those upper and lower angles'''
    
    acos_z_u_escaped = np.array([np.arccos(zu) for zu in z_u_escaped])
    angle_indices = np.where( ((acos_z_u_escaped >= theta_lower) & (acos_z_u_escaped < theta_upper)) | ((acos_z_u_escaped >= theta_plus_lower) & (acos_z_u_escaped < theta_plus_upper)))[0]

    return x_frec_escaped[angle_indices] 

In [4]:
# Main functions

def get_unitary_vectors(x_sphere, y_sphere, z_sphere, R):
    '''Unitary vectors in the x, y, z directions'''
    x_unitary = x_sphere/R
    y_unitary = y_sphere/R
    z_unitary = z_sphere/R
    
    return x_unitary, y_unitary, z_unitary

def get_atom_velocities(x_unitary, y_unitary, z_unitary, vrot):
    '''Defines new atom velocities based on vrot (vr)'''
    vrot = vrot*0.5 
    v_x = - y_unitary*vrot
    v_y = x_unitary*vrot
    v_z = np.zeros(len(z_unitary))
    
    return v_x, v_y, v_z

def doppler_shift(x_unitary, y_sphere, z_u_escaped, x_frec_escaped, v_x, v_y, v_z, theta_lower, theta_upper, theta_plus_lower, theta_plus_upper, phis, v_th):
    '''Gets the final velocities after the doppler shift'''
    
    final_velocities = []

    acos_x_escaped_pos = np.array([np.arccos(xu) for xu in x_unitary])
    acos_x_escaped_neg = np.array([2.0*np.pi-np.arccos(xu) for xu in x_unitary])
    acos_z_u_escaped = np.array([np.arccos(zu) for zu in z_u_escaped])
    
    for n_phi in range(len(phis)-1):
        phi_lower = phis[n_phi]
        phi_upper = phis[n_phi+1]
        phi_middle = (phi_lower + phi_upper)/2.0
        
        r_observer = np.array([np.cos(phi_middle), np.sin(phi_middle), 0])

        angle_indices = np.where( ((y_sphere >= 0) & (acos_x_escaped_pos >= phi_lower) & (acos_x_escaped_pos < phi_upper)) | ((y_sphere < 0) & (acos_x_escaped_neg >= phi_lower) & (acos_x_escaped_neg < phi_upper)) | ((acos_z_u_escaped >= theta_lower) & (acos_z_u_escaped < theta_upper)) | ((acos_z_u_escaped >= theta_plus_lower) & (acos_z_u_escaped < theta_plus_upper)) )[0]

        x_frec_escaped_angle = x_frec_escaped[angle_indices]

        v_x_angle = v_x[angle_indices]
        v_y_angle = v_y[angle_indices]
        v_z_angle = v_z[angle_indices]

        shift_factors_angle = v_x_angle*r_observer[0] + v_y_angle*r_observer[1] + v_z_angle*r_observer[2]

        shifted_velocities = x_frec_escaped_angle*v_th + shift_factors_angle

        final_velocities = np.concatenate((final_velocities, shifted_velocities))
        
    return final_velocities

In [5]:
# Plots

def plot_final_positions(x_escaped, y_escaped, z_escaped):

    fig = plt.figure(figsize=(25,25))
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(x_escaped, y_escaped, z_escaped, s=1, lw = 0)

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')

    plt.show()
    
def plot_final_positions_sphere(x_escaped_sphere, y_escaped_sphere, z_escaped_sphere, x_u_escaped, y_u_escaped, z_u_escaped):
    
    fig = plt.figure(figsize=(25,25))
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(x_escaped_sphere, y_escaped_sphere, z_escaped_sphere, s=1, lw = 0, c='c')
    ax.quiver(x_escaped_sphere[0:500], y_escaped_sphere[0:500], z_escaped_sphere[0:500], x_u_escaped[0:500], y_u_escaped[0:500], z_u_escaped[0:500], length=1E11)

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')

    plt.show()
    
def plot_spectrum(final_velocities, vrot, vout, logtau, theta_lower, theta_upper, plot_non_doppler=False):
    
    fig = plt.figure(1, (10., 6.))

    props = dict(boxstyle='square', facecolor='white')    

    vrot_lab = r'${0:.0f}$'.format(vrot)
    vout_lab = r'${0:.0f}$'.format(vout)
    theta_lower_lab = r'${0:.0f}$'.format(int(np.rad2deg(theta_lower)))
    theta_upper_lab = r'${0:.0f}$'.format(int(np.rad2deg(theta_upper)))

    theta_lab = r'$\theta=$ '
    lab = '$v_{rot}=$ '+vrot_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$\n $v_{out}=$ '+vout_lab+' $\mathrm{km}$ $\mathrm{s^{-1}}$'

    n, b = np.histogram(final_velocities, bins=40)        
    delta_v = b[1]-b[0]
    area = delta_v * sum(n)
    n = n/area

    border_x = []
    border_y = []

    for j in range(len(n)-1):
        border_x.append(b[j]+delta_v/2)
        border_x.append(b[j]+delta_v/2)
        border_y.append(n[j])
        border_y.append(n[j+1])

    plt.bar(b[0:40], n, width=delta_v, color='c', edgecolor=None, alpha = 1)
    plt.plot(border_x, border_y, c='k', linewidth=1)

    plt.axvline(x=0, ymin=0, ymax=1, c='k', linestyle='--', linewidth=1)
    plt.xlim(-200,250)
    plt.ylim(0,0.013)
    plt.ylabel('$\mathrm{Intensity}$')
    plt.xlabel('$\mathrm{V}$ ($\mathrm{km}$ $\mathrm{s^{-1}}$)')
    plt.text(-180,0.011, lab, fontsize=20, bbox=props)
    
    tau_dir = 'tau10E'+str(logtau)
    vrot_dir = 'vrot'+str(vrot)
    vout_dir = 'vout'+str(vout)
    fname = tau_dir+'/'+tau_dir+'_'+vrot_dir+'_'+vout_dir
    if plot_non_doppler:
        fname += '_simulated.png'
    else:
        fname += '_doppler_shifted.png'

    plt.savefig(fname, format='png', transparent=False, bbox_inches='tight')
    plt.close()

### Values

In [6]:
# Combination}s

logtaus = [5, 6, 7]
#logtaus = [5]
vrot0 = 0
vrots = [50, 100]
#vrots = [50]
vouts = [25, 50] #[25, 50, 75]
#vouts = [25] #[25, 50, 75]

# Fixed values
num_thetas = 9
num_phis = 180

v_th = -12.8

### Execution

In [7]:
# Same angles for all combinations 
thetas, thetas_plus, phis = get_angles(num_thetas, num_phis)

# Getting angle ~90°
theta_lower = thetas[-2] 
theta_upper = thetas[-1]
theta_plus_lower = thetas_plus[-1]
theta_plus_upper = thetas_plus[-2]

In [8]:
# Looping through combinations

for logtau in logtaus:
    for vrot in vrots:
        for vout in vouts:
            
            # read data
            x_escaped, y_escaped, z_escaped, x_u_escaped, y_u_escaped, z_u_escaped, x_frec_escaped = read_data(vrot0, vout, logtau)
            num_photons_escaped = len(x_escaped)
            
            # sphere radius
            R = get_radius(x_escaped, y_escaped, z_escaped)
            #R = np.sqrt( x_escaped**2 + y_escaped**2 + z_escaped**2 )
            
            # positions in sphere
            x_sphere, y_sphere, z_sphere = get_final_positions_sphere(x_escaped, y_escaped, z_escaped, x_u_escaped, y_u_escaped, z_u_escaped, R)
            #x_sphere, y_sphere, z_sphere = x_escaped, y_escaped, z_escaped
            
            # unitary vectors
            x_unitary, y_unitary, z_unitary = get_unitary_vectors(x_sphere, y_sphere, z_sphere, R)
            
            # atom velocities
            v_x, v_y, v_z = get_atom_velocities(x_unitary, y_unitary, z_unitary, vrot)
            
            # doppler shift
            final_velocities = doppler_shift(x_unitary, y_sphere, z_u_escaped, x_frec_escaped, v_x, v_y, v_z, theta_lower, theta_upper, theta_plus_lower, theta_plus_upper, phis, v_th)
            
            # plot doppler shifted spectrum
            plot_spectrum(final_velocities, vrot, vout, logtau, theta_lower, theta_upper)
            
            # get and plot spectrum from simulation (vout)
            x_sim, y_sim, z_sim, x_u_sim, y_u_sim, z_u_sim, x_frec_sim = read_data(vrot, vout, logtau)
            x_frec_sim_thetas = filter_by_theta(z_u_sim, x_frec_sim, theta_lower, theta_upper, theta_plus_lower, theta_plus_upper)
            final_velocities_sim = x_frec_sim_thetas*v_th
            plot_spectrum(final_velocities_sim, vrot, vout, logtau, theta_lower, theta_upper, plot_non_doppler=True)
            
            # get and plot spectrum from simulation (vout0)
            x_sim0, y_sim0, z_sim0, x_u_sim0, y_u_sim0, z_u_sim0, x_frec_sim0 = read_data(vrot0, vout, logtau)
            x_frec_sim_thetas0 = filter_by_theta(z_u_sim0, x_frec_sim0, theta_lower, theta_upper, theta_plus_lower, theta_plus_upper)
            final_velocities_sim0 = x_frec_sim_thetas0*v_th
            plot_spectrum(final_velocities_sim0, vrot0, vout, logtau, theta_lower, theta_upper, plot_non_doppler=True)